# 1. Preparación del entorno

In [1]:
# Install
!pip install tabula-py
!pip install pyPDF2
# Import
import tabula as tb
import pandas as pd
import requests
import base64
import json
from datetime import datetime, timedelta
import time
import os
import PyPDF2
import re

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.4 MB/s eta 0:00:00


# 2. Descarga de PDFs de BORA
BORA genera PDFs de los boletines de cada día.
La primera sección es la que hace mención a Legislación y avisos oficiales

In [7]:
class BoletinDownloader:
    def __init__(self, download_folder="boletin_pdfs"):
        self.session = requests.Session()
        self.base_url = "https://www.boletinoficial.gob.ar"
        self.download_url = f"{self.base_url}/pdf/download_section"
        self.download_folder = download_folder

        # Create download folder if it doesn't exist
        os.makedirs(download_folder, exist_ok=True)

        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
            'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
            'X-Requested-With': 'XMLHttpRequest',
            'Origin': self.base_url,
            'Referer': f'{self.base_url}/'
        }

    def download_section(self, section, date):
        """Download a specific section for a given date"""
        date_str = date.strftime("%Y%m%d")
        filename = f"seccion_{section}_{date_str}.pdf"
        filepath = os.path.join(self.download_folder, filename)

        # Check if file already exists
        if os.path.exists(filepath):
            print(f"✓ File already exists: {filename}")
            return True

        data = {'nombreSeccion': section}

        try:
            response = self.session.post(
                self.download_url,
                data=data,
                headers=self.headers,
                timeout=30
            )

            if response.status_code == 200:
                json_data = response.json()

                if 'pdfBase64' in json_data and json_data['pdfBase64']:
                    pdf_data = base64.b64decode(json_data['pdfBase64'])

                    with open(filepath, 'wb') as f:
                        f.write(pdf_data)

                    print(f"✓ Downloaded: {filename} ({len(pdf_data)} bytes)")
                    return True
                else:
                    print(f"✗ No PDF available for {date_str}")
                    return False
            else:
                print(f"✗ HTTP {response.status_code} for {date_str}")
                return False

        except requests.exceptions.RequestException as e:
            print(f"✗ Network error for {date_str}: {e}")
            return False
        except json.JSONDecodeError as e:
            print(f"✗ JSON decode error for {date_str}: {e}")
            return False
        except Exception as e:
            print(f"✗ Unexpected error for {date_str}: {e}")
            return False


# test with today's date
downloader = BoletinDownloader()
test_date = datetime.now()
downloader.download_section("primera", test_date)

✓ Downloaded: seccion_primera_20251021.pdf (2842359 bytes)


True

# 3. Implementando Bag of words
Bag of words es un modelo muy utilizado en text mining.
Tiene la particularidad de adaptarse bien a corpus textuales muy diversos permitiendo describir el corpus sin ser especialista en el dominio

## 3.1 Preparar dataset e importarinstalar librerias

In [15]:
# import and install

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
spanish_stopwords = stopwords.words('spanish')
from sklearn.feature_extraction.text import CountVectorizer
import spacy
import locale
import string
locale.getpreferredencoding = lambda: "UTF-8"
! spacy download es_core_news_sm
nlp = spacy.load("es_core_news_sm")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 61.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [17]:
# Create a dataframe organized in paragraphs
def extract_paragraphs_to_dataframe(pdf_path):
    """
    Extract text at paragraph level for better context
    """
    paragraphs_data = []

    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)

        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text = page.extract_text()

            # Split into paragraphs
            page_paragraphs = [p.strip() for p in text.split('\n\n') if p.strip()]

            for para_num, paragraph in enumerate(page_paragraphs):
                paragraphs_data.append({
                    'page': page_num + 1,
                    'paragraph_id': f"p{page_num+1}_{para_num+1}",
                    'text': paragraph,
                    'char_count': len(paragraph),
                    'word_count': len(paragraph.split())
                })

    return pd.DataFrame(paragraphs_data)

# Usage
pdf_path = "boletin_pdfs/seccion_primera_20251021.pdf"
df_paragraphs = extract_paragraphs_to_dataframe(pdf_path)
print(df_paragraphs.head())

   page paragraph_id                                               text  \
0     1         p1_1  Registro Nacional de la Propiedad Intelectual ...   
1     2         p2_1  BOLETÍN OFICIAL Nº 35.774 - Primera Sección 2 ...   
2     3         p3_1  BOLETÍN OFICIAL Nº 35.774 - Primera Sección 3 ...   
3     4         p4_1  BOLETÍN OFICIAL Nº 35.774 - Primera Sección 4 ...   
4     5         p5_1  BOLETÍN OFICIAL Nº 35.774 - Primera Sección 5 ...   

   char_count  word_count  
0         827         117  
1        5789         392  
2        2239          82  
3        3367         508  
4        5532         812  


## 3.2 Funciones para BOW

In [20]:
def preprocess_spanish_text(text):
    """
    Preprocess a single Spanish text with lemmatization
    """
    if pd.isna(text) or text == '':
        return ""

    doc = nlp(text.lower())

    lemmas = []
    for token in doc:
        if (not token.is_stop and
            not token.is_punct and
            not token.is_space and
            len(token.text) > 2 and
            token.is_alpha):
            lemmas.append(token.lemma_)

    return " ".join(lemmas)

def apply_text_preprocessing(df, text_column):
    """
    Apply preprocessing to a DataFrame column
    Returns a new DataFrame with processed text
    """
    df_processed = df.copy()
    df_processed['processed_text'] = df_processed[text_column].apply(preprocess_spanish_text)
    df_processed['original_word_count'] = df_processed[text_column].str.split().str.len()
    df_processed['processed_word_count'] = df_processed['processed_text'].str.split().str.len()

    return df_processed

def calculate_text_statistics(df, original_col, processed_col):
    """
    Calculate text statistics for comparison
    """
    total_original = df[original_col].str.split().str.len().sum()
    total_processed = df[processed_col].str.split().str.len().sum()

    all_original = ' '.join(df[original_col].astype(str))
    all_processed = ' '.join(df[processed_col].astype(str))

    unique_original = len(set(all_original.lower().split()))
    unique_processed = len(set(all_processed.split()))

    stats = {
        'total_paragraphs': len(df),
        'total_original_words': total_original,
        'total_processed_words': total_processed,
        'unique_original_words': unique_original,
        'unique_processed_words': unique_processed,
        'word_reduction_pct': ((total_original - total_processed) / total_original * 100),
        'vocab_reduction_pct': ((unique_original - unique_processed) / unique_original * 100)
    }

    return stats

def create_bow_analysis(text_series, top_n=20, **vectorizer_kwargs):
    """
    Create Bag-of-Words analysis from a text series
    """
    # Combine all text
    all_text = ' '.join(text_series.astype(str))

    # Default vectorizer parameters
    default_params = {
        'max_features': top_n * 2,
        'lowercase': True,
        'token_pattern': r'\b[a-zA-ZáéíóúñÁÉÍÓÚÑ]+\b'
    }
    default_params.update(vectorizer_kwargs)

    vectorizer = CountVectorizer(**default_params)
    bow_matrix = vectorizer.fit_transform([all_text])

    # Get word frequencies
    word_freq = dict(zip(vectorizer.get_feature_names_out(), bow_matrix.sum(axis=0).tolist()[0]))
    top_words = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)[:top_n]

    return pd.DataFrame(top_words, columns=['word', 'frequency'])


def print_preprocessing_report(df, original_col='text', processed_col='processed_text'):
    """
    Print a comprehensive preprocessing report
    """
    stats = calculate_text_statistics(df, original_col, processed_col)

    print("="*60)
    print("TEXT PREPROCESSING REPORT")
    print("="*60)

    print(f"Total paragraphs: {stats['total_paragraphs']:,}")
    print(f"Total words - Original: {stats['total_original_words']:,}")
    print(f"Total words - Processed: {stats['total_processed_words']:,}")
    print(f"Word reduction: {stats['word_reduction_pct']:.1f}%")
    print(f"Unique words - Original: {stats['unique_original_words']:,}")
    print(f"Unique words - Processed: {stats['unique_processed_words']:,}")
    print(f"Vocabulary reduction: {stats['vocab_reduction_pct']:.1f}%")

    return stats

def print_sample_transformations(df, original_col='text', processed_col='processed_text', sample_size=3):
    """
    Print sample transformations for quality check
    """
    print("\n" + "="*60)
    print("SAMPLE TRANSFORMATIONS")
    print("="*60)

    for i in range(min(sample_size, len(df))):
        original = df.iloc[i][original_col]
        processed = df.iloc[i][processed_col]

        # Try to get page number if available
        page_info = f"Page {df.iloc[i]['page']}" if 'page' in df.columns else f"Row {i+1}"

        print(f"\nSample {i+1} ({page_info}):")
        print(f"ORIGINAL: {original[:150]}...")
        print(f"PROCESSED: {processed[:150]}...")

        if 'original_word_count' in df.columns and 'processed_word_count' in df.columns:
            print(f"Words: {df.iloc[i]['original_word_count']} → {df.iloc[i]['processed_word_count']}")


def process_paragraphs_dataframe(df_paragraphs, text_column='text'):
    """
    Specific function to process your paragraphs DataFrame
    """
    print("Applying lemmatization to paragraphs...")
    df_processed = apply_text_preprocessing(df_paragraphs, text_column)

    # Filter out empty processed paragraphs
    df_processed = df_processed[df_processed['processed_text'].str.len() > 0].copy()

    print(f"Original paragraphs: {len(df_paragraphs)}")
    print(f"Paragraphs after processing: {len(df_processed)}")

    return df_processed

def analyze_paragraphs_bow(df_processed, text_column='processed_text', top_n=20):
    """
    Specific function to analyze preprocessed paragraphs with BOW
    """
    print("\n" + "="*60)
    print("BAG OF WORDS ANALYSIS - PROCESSED PARAGRAPHS")
    print("="*60)

    top_words_df = create_bow_analysis(df_processed[text_column], top_n=top_n)

    print(f"Top {len(top_words_df)} most frequent lemmas:")
    print(top_words_df.to_string(index=False))

    return top_words_df


def run_paragraphs_analysis_pipeline(df_paragraphs, text_column='text', top_n=20):
    """
    Complete pipeline for analyzing your paragraphs data
    """
    # Step 1: Preprocess data
    df_processed = process_paragraphs_dataframe(df_paragraphs, text_column)

    # Step 2: Generate reports
    stats = print_preprocessing_report(df_processed)
    print_sample_transformations(df_processed)

    # Step 3: BOW analysis
    top_words = analyze_paragraphs_bow(df_processed, top_n=top_n)

    return df_processed, top_words, stats

# =============================================================================
# USAGE
# =============================================================================

# Complete pipeline (your main use case)
df_processed, top_words, stats = run_paragraphs_analysis_pipeline(df_paragraphs)

# Just preprocessing
# df_processed = process_paragraphs_dataframe(df_paragraphs)

# Just BOW analysis on already processed data
#top_words = analyze_paragraphs_bow(df_processed)

# Compare original vs processed BOW
print("\n" + "="*60)
print("COMPARISON: ORIGINAL vs PROCESSED BOW")
print("="*60)

bow_original = create_bow_analysis(df_processed['text'], top_n=10)
bow_processed = create_bow_analysis(df_processed['processed_text'], top_n=10)

print("Top 10 words - ORIGINAL:")
print(bow_original.to_string(index=False))

print("\nTop 10 words - PROCESSED (lemmatized):")
print(bow_processed.to_string(index=False))

Applying lemmatization to paragraphs...
Original paragraphs: 99
Paragraphs after processing: 99
TEXT PREPROCESSING REPORT
Total paragraphs: 99
Total words - Original: 61,072
Total words - Processed: 27,114
Word reduction: 55.6%
Unique words - Original: 7,069
Unique words - Processed: 2,979
Vocabulary reduction: 57.9%

SAMPLE TRANSFORMATIONS

Sample 1 (Page 1):
ORIGINAL: Registro Nacional de la Propiedad Intelectual Nº 5.218.874
DOMICILIO LEGAL: Suipacha 767 - C1008AAO
Ciudad Autónoma de Buenos Aires
Tel. 5218-8400Los ...
PROCESSED: registro nacional propiedad intelectual domicilio legal suipachaber ciudad autónomo aires tel documento aparecer boletín oficial república argentino t...
Words: 117 → 66

Sample 2 (Page 2):
ORIGINAL: BOLETÍN OFICIAL Nº 35.774 - Primera Sección 2 Martes 21 de octubre de 2025
SUMARIO
Avisos Nuevos
Leyes
LEY DE FINANCIAMIENTO DE LA EDUCACIÓN UNIVERSIT...
PROCESSED: boletín oficial sección martes octubre sumario aviso ley ley financiamiento educación universitar

# 4. Otros

Algunos scripts para explorar el contenido de los PDFs

In [ ]:
import tabula
import pandas as pd
import re
from collections import Counter
import requests

# Since we're in Colab, let's work with the downloaded PDF file
pdf_path = "boletin_pdfs/seccion_primera_20250927.pdf"

# First, let's get some basic information about the PDF
# Get the number of pages
import PyPDF2
with open(pdf_path, 'rb') as file:
    pdf_reader = PyPDF2.PdfReader(file)
    num_pages = len(pdf_reader.pages)
    print(f"Number of pages in the PDF: {num_pages}")

# Now let's extract all the text to search for "derechos humanos"
full_text = ""
for page_num in range(num_pages):
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        page = pdf_reader.pages[page_num]
        full_text += page.extract_text()

# Count occurrences of "derechos humanos"
derechos_humanos_count = full_text.lower().count("derechos humanos")
print(f"Occurrences of 'derechos humanos': {derechos_humanos_count}")

# Now let's use tabula to extract tables and look for resoluciones and disposiciones
# Let's try to extract all tables from the PDF
tables = tabula.read_pdf(pdf_path, pages='all', multiple_tables=True)
print(f"Number of tables found: {len(tables)}")

# Let's examine the structure of the first few tables to understand the content
for i, table in enumerate(tables[:5]):
    print(f"\nTable {i+1} shape: {table.shape}")
    print(f"Table {i+1} columns: {table.columns.tolist()}")
    if not table.empty:
        print(table.head(3))

In [22]:
# Listar resoluciones

# Extract text from page 2 specifically
with open(pdf_path, 'rb') as file:
    pdf_reader = PyPDF2.PdfReader(file)
    page2_text = pdf_reader.pages[1].extract_text()  # Page index 1 is page 2

print("Content from page 2 (SUMARIO):")
print(page2_text[:2000])  # First 2000 characters

# Let's search for resolution patterns in page 2 specifically
resoluciones_page2 = re.findall(resoluciones_pattern, page2_text, re.IGNORECASE)
print(f"\nResoluciones found in SUMARIO page: {len(set(resoluciones_page2))}")
for resol in sorted(set(resoluciones_page2)):
    print(f"  - {resol}")

# Let's also check if we can find the specific sections for Resoluciones and Disposiciones
# by looking at the table of contents structure

Content from page 2 (SUMARIO):
 BOLETÍN OFICIAL Nº 35.774 - Primera Sección 2 Martes 21 de octubre de 2025
SUMARIO
Avisos Nuevos
Leyes
LEY DE FINANCIAMIENTO DE LA EDUCACIÓN UNIVERSITARIA Y RECOMPOSICIÓN DEL SALARIO DOCENTE. Ley 27795. Disposiciones.  ................  4
Decreto 759/2025. DECTO-2025-759-APN-PTE - Promúlgase la Ley N° 27.795.  ........................................................................................................  6
EMERGENCIA SANITARIA DE LA SALUD PEDIÁTRICA Y DE LAS RESIDENCIAS NACIONALES EN SALUD. Ley 27796. Disposiciones.  .......................  11
Decreto 760/2025. DECTO-2025-760-APN-PTE - Promúlgase la Ley N° 27.796.  ........................................................................................................  12
Resoluciones
DIRECCIÓN NACIONAL DE VIALIDAD. Resolución 1619/2025. RESOL-2025-1619-APN-DNV#MEC.  ..........................................................................  17
DIRECCIÓN NACIONAL DE VIALIDAD. Resolución 1620/2

In [ ]:
# Dataframe de prueba
# Cambiar el pdf path al que se haya descargado en 📁
pdf_path = "boletin_pdfs/seccion_primera_20251020.pdf"
with open(pdf_path, 'rb') as file:
    pdf_reader = PyPDF2.PdfReader(file)
    num_pages = len(pdf_reader.pages)
    print(f"Number of pages in the PDF: {num_pages}")

full_text = ""
for page_num in range(num_pages):
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        page = pdf_reader.pages[page_num]
        full_text += page.extract_text()
df_test =  full_text

Number of pages in the PDF: 124
